# Exploring eBay Car Sales Data

In this project, I'll be analyzing a dataset of used cars from *eBay Kleinanzeigen*, a classifieds section of the German eBay website. The dataset was originally scraped and uploaded to Kaggle by user [orgesleka](https://www.kaggle.com/orgesleka). Though it is no longer available on Kaggle, it can be accessed [here](https://data.world/data-society/used-cars-data) via data.world.

There are a couple of modications made to the original dataset:
* The dataset has been sampled down to 50,000 data points (from 370,000 total used cars) for the sake of efficient processing for this exercise.
* The dataset has been intentionally "dirtied" so as to be cleaned as a part of this exercise (simulating a less sanitized result of a scraped dataset).

Here's a breakdown of the data dictionary / schema for the dataset (descriptions taken from [DataQuest.io](dataquest.io):
* `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
* `name` - Name of the car.
* `seller` - Whether the seller is private or a dealer.
* `offerType` - The type of listing
* `price` - The price on the ad to sell the car.
* `abtest` - Whether the listing is included in an A/B test.
* `vehicleType` - The vehicle Type.
* `yearOfRegistration` - The year in which the car was first registered.
* `gearbox` - The transmission type.
* `powerPS` - The power of the car in PS.
* `model` - The car model name.
* `kilometer` - How many kilometers the car has driven.
* `monthOfRegistration` - The month in which the car was first registered.
* `fuelType` - What type of fuel the car uses.
* `brand` - The brand of the car.
* `notRepairedDamage` - If the car has a damage which is not yet repaired.
* `dateCreated` - The date on which the eBay listing was created.
* `nrOfPictures` - The number of pictures in the ad.
* `postalCode` - The postal code for the location of the vehicle.
* `lastSeenOnline` - When the crawler saw this ad last online.

I'll make use of the pandas and numpy libraries for python to facilitate cleaning and analyzing the data.

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [2]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


After showing some quick summaries of the data, we can already see that some fields are represented as strings/objects when numerical representations may be more useful (e.g. prices, power, odometer readings, dates, etc.) and that some fields are represented in German. Likewise, we can note that there are null values in some columns, though no columns have more than ~20% null values.


## Cleaning columns

First, let's convert the column names from snakeCase to camel_case, our preferred naming convention, and modify some column names for readability.

In [3]:
autos.rename(columns={
    'yearOfRegistration': 'registration_year',
    'monthOfRegistration': 'registration_month',
    'notRepairedDamage': 'unrepaired_damage',
    'dateCreated': 'ad_created',
    'dateCrawled': 'date_crawled',
    'offerType': 'offer_type',
    'vehicleType': 'vehicle_type',
    'gearBox': 'gear_box',
    'powerPS': 'power_PS',
    'fuelType': 'fuel_type',
    'nrOfPictures': 'nr_of_pictures',
    'postalCode': 'postal_code',
    'lastSeen': 'last_seen'   
}, inplace=True)
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Initial exploration and cleaning

We'll kick off with some basic review and cleaning of the data.

In [4]:
# Generate some brief descriptive statistics
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-22 09:51:06,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Some quick observations: the number of pictures (`nr_of_pictures`) column doesn't seem to have any useful data (i.e. lists 0.0 pictures in all cases). There are also certain columns with only two possible values: `seller` and `offer_type`. Likewise, we have the aforementioned columns with numeric data represented as text.

In [5]:
# Investigate seller and offer_type columns further
print(autos['seller'].value_counts(), '\n')
print(autos['offer_type'].value_counts())

# Remove the num_photos column
# autos = autos.drop('num_photos', axis=1)

privat        49999
gewerblich        1
Name: seller, dtype: int64 

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64


It seems that these columns are almost entirely showing a single value respectively. Translations yield that "privat" listings are "private" (perhaps an individual has posted the listing) and "gewerblich" listings are "commercial" (perhaps a listing for business purposes). "Angebot" refers to an "offer", whereas "gesuch" refers to a "wanted" car. There's a bit more of a semantic difference there (i.e. a wanted car likely doesn't actually exist for sale). We'll leave these rows and columns in for now, but remove the `num_photos` columns.

In [6]:
# Remove non-numeric characters from price and odometer columns, convert to numeric dtype
autos['price'] = (autos['price']
                          .str.replace("$","")
                          .str.replace(",","")
                          .astype(float)
                          )
autos['odometer'] = (autos['odometer']
                         .str.replace('km', '')
                         .str.replace(',', '')
                         .astype(int)
                    )

# Rename odometer column to odometer_km to denote measurement unit
autos.rename(columns={'odometer': 'odometer_km'}, inplace=True)
autos.loc[:5,['price', 'odometer_km']]

,price,odometer_km
0,5000.0,150000
1,8500.0,150000
2,8990.0,70000
3,4350.0,70000
4,1350.0,150000
5,7900.0,150000


## Focus on Odometer and Price data

Next, we'll explore our odometer and price data a bit more deeply to find outliers and erroneous looking data points that might distort our analysis.

In [7]:
# Display the count of unique values
autos['odometer_km'].unique().shape

(13,)

In [8]:
# Display common statistical measures
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [9]:
# Display counts for odometer reading buckets
autos['odometer_km'].value_counts().sort_index(ascending=False)

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
40000       819
30000       789
20000       784
10000       264
5000        967
Name: odometer_km, dtype: int64

Based on these summaries, it seems that the odometer data must have a few attributes:
1. It's been simplified- the minimum value is a round 5,000, for example, and the data on the whole seems to be rounded to the (nearest?) thousand km.
2. The data skews heavily toward the higher end of mileage- we might expect more mileage from used cars generally, but 50% of the used cars read at 150,000km, which is also the max value, and 75% at at least 125,000km. 
3. With such a high percentage of the dataset at 150,000km, we might assume that this represents a virtual maximum, even though mileages might be higher.
With these attributes in mind, we might assume that used car listers were picking mileages from a drop-down / picklist.

In [10]:
# Display the count of unique values for price
autos['price'].unique().shape

(2357,)

In [11]:
# Display common statistical measures for price
autos['price'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [12]:
# Display counts for high-end price buckets
autos['price'].value_counts().sort_index(ascending=False).head(15)

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
Name: price, dtype: int64

In [13]:
# Display counts for low-end price buckets
autos['price'].value_counts().sort_index(ascending=True).head(15)

0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
13.0       2
14.0       1
15.0       2
17.0       3
18.0       1
Name: price, dtype: int64

Some observations from this analysis- some cars are priced unrealistically exorbitantly (\\$100M and others negligibly - free to \\$18). Prices are likewise rounded to the nearest dollar. Given that eBay supports auctions, low starting prices would probably make sense, but they wouldn't lend themselves well to approximating a relative value for cars (in fact, any analysis of relative value would probably not be a great idea for this dataset without being able to know final sale prices). We'll take a look at how many cars fit into this "negligibly low" bucket.

In [14]:
print(autos['price'].between(1,99).value_counts(), '\n')
print(autos['price'].between(99,1000).value_counts())

False    49659
True       341
Name: price, dtype: int64 

False    39744
True     10256
Name: price, dtype: int64


It seems that only 341 additional cars (beyond the ~1500 cars at free to \\$1) are priced below \\$100, whereas the next order of magnitude up encompasses a full 20% of our dataset. Given the lower number of otherwise cheap cars and the higher number of free and \\$1 cars, we'll remove these rows for the remainder of our analysis to avoid further distortion to the analysis.

For the higher prices, there seems to be a large jump above the car priced at \\$350,000 to the next order of magnitude and then beyond, so we'll do the same for these cars.

In [15]:
autos = autos[autos['price'].between(100, 350000)]
autos['price'].describe()

count     48224.000000
mean       5930.371433
std        9078.372762
min         100.000000
25%        1250.000000
50%        3000.000000
75%        7499.000000
max      350000.000000
Name: price, dtype: float64

## Exploring dates

As previously mentioned, our dates are represented as strings, as opposed to a more useful numeric format. We'll hop into those next.

Here are the date-oriented columns:
* `date_crawled`
* `registration_month`
* `registration_year`
* `ad_created`
* `last_seen`

Two refer to the web crawler's behavior, one refers to the listing (these three are string-based timestamps), two to the vehicle itself.

In [16]:
# Display summary on timestamp date columns
autos[['date_crawled', 'ad_created', 'last_seen']][:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [17]:
# Summarize distribution of date originally crawled
(autos['date_crawled']
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
)

2016-03-05    0.025361
2016-03-06    0.014039
2016-03-07    0.036061
2016-03-08    0.033179
2016-03-09    0.033013
2016-03-10    0.032287
2016-03-11    0.032598
2016-03-12    0.036911
2016-03-13    0.015677
2016-03-14    0.036662
2016-03-15    0.034319
2016-03-16    0.029467
2016-03-17    0.031499
2016-03-18    0.012898
2016-03-19    0.034734
2016-03-20    0.037803
2016-03-21    0.037201
2016-03-22    0.032888
2016-03-23    0.032287
2016-03-24    0.029446
2016-03-25    0.031499
2016-03-26    0.032308
2016-03-27    0.031126
2016-03-28    0.034962
2016-03-29    0.034112
2016-03-30    0.033738
2016-03-31    0.031851
2016-04-01    0.033697
2016-04-02    0.035605
2016-04-03    0.038611
2016-04-04    0.036538
2016-04-05    0.013064
2016-04-06    0.003173
2016-04-07    0.001389
Name: date_crawled, dtype: float64

In [18]:
# Summarize distribution of last date seen by crawler
(autos['last_seen']
         .str[:10]
         .value_counts(normalize=True, dropna=False)
         .sort_index()
)

2016-03-05    0.001078
2016-03-06    0.004313
2016-03-07    0.005433
2016-03-08    0.007320
2016-03-09    0.009580
2016-03-10    0.010638
2016-03-11    0.012400
2016-03-12    0.023785
2016-03-13    0.008875
2016-03-14    0.012629
2016-03-15    0.015863
2016-03-16    0.016444
2016-03-17    0.028098
2016-03-18    0.007320
2016-03-19    0.015760
2016-03-20    0.020654
2016-03-21    0.020550
2016-03-22    0.021359
2016-03-23    0.018580
2016-03-24    0.019762
2016-03-25    0.019098
2016-03-26    0.016672
2016-03-27    0.015552
2016-03-28    0.020840
2016-03-29    0.022292
2016-03-30    0.024697
2016-03-31    0.023826
2016-04-01    0.022852
2016-04-02    0.024884
2016-04-03    0.025133
2016-04-04    0.024531
2016-04-05    0.125062
2016-04-06    0.221964
2016-04-07    0.132154
Name: last_seen, dtype: float64

Given the even spacing of dates and distribution, we can infer some crawler behavior- daily crawling, relatively even number of crawling of posts any given day. The number of posts last seen picks up appreciably toward the end. Ostensibly, posts are piling up over time. Drops in postings probably indicate that a listing was removed- either sold or taken down, likely.

In [19]:
# Summarize distribution of dates auto ads/listings were created
(autos['ad_created']
         .str[:10]
         .value_counts(normalize=True, dropna=False)
         .sort_index()
)

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038860
2016-04-04    0.036890
2016-04-05    0.011799
2016-04-06    0.003256
2016-04-07    0.001244
Name: ad_created, Length: 76, dtype: float64

There's a good bit of variation in listing dates. It makes sense that we'd see fewer old posts because those listings would be removed over time- either sold or taken down.

In [20]:
# Display distribution of the car's registration
autos['registration_year'].describe()

count    48224.000000
mean      2004.730964
std         87.897388
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

There's some odd data here- cars wouldn't be registered before the invention of cars, nor in the future. Likely some user error here in the listings, but perhaps the site should've applied some boundaries on options for registration year.

We'll take a further look at the values and apply these parameters ourselves- ideally the years should fall between the invention of the automobile (we'll give listers the benefit of the doubt and cut-off at the [first working "car"](https://en.wikipedia.org/wiki/Car#History)) and the year that the listing was crawled (2016).

In [21]:
print(autos['registration_year'].between(1950,9999).value_counts(), '\n')
print(autos['registration_year'].between(1881,2016).value_counts(), '\n')
print(autos['registration_year'].between(2200,9999).value_counts(), '\n')
print(autos['registration_year'].between(2016,2200).value_counts(), '\n')

print('ratio legitimate dated registration: ', (autos['registration_year'].between(1881,2016).sum() 
                                        / autos['registration_year'].shape[0]))

True     48202
False       22
Name: registration_year, dtype: int64 

True     46352
False     1872
Name: registration_year, dtype: int64 

False    48210
True        14
Name: registration_year, dtype: int64 

False    45169
True      3055
Name: registration_year, dtype: int64 

ratio legitimate dated registration:  0.9611811546118115


It looks as if there's not much of a difference between a very reasonable date and this first date on the lower bound. It seems that the majority of erroneous dates are in the near future so this might lead us to drop the very early (pre-1881 and very late (post-2200) dates. It's not really clear what's to be gained by dropping this data, though. It's probably worth another look into the underlying process here- are users reporting dates? Is there a system scanning registration documents? Is the implication that incorrect dates speak to illegitimate listings? In this case, we'll be safe and drop any date not within a generally logical date, which looks to represent less than 4% of our dataset.

In [22]:
autos = autos[autos['registration_year'].between(1881, 2016)]
print(autos['registration_year'].value_counts(normalize=True), '\n')
print(autos['registration_year'].value_counts(normalize=True).head(10))
autos['registration_year'].describe()

2000    0.066966
2005    0.062802
1999    0.062112
2004    0.058228
2003    0.058099
          ...   
1938    0.000022
1939    0.000022
1953    0.000022
1943    0.000022
1952    0.000022
Name: registration_year, Length: 78, dtype: float64 

2000    0.066966
2005    0.062802
1999    0.062112
2004    0.058228
2003    0.058099
2006    0.057560
2001    0.056718
2002    0.053439
1998    0.050483
2007    0.049038
Name: registration_year, dtype: float64


count    46352.000000
mean      2002.939787
std          7.127146
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

This distribution makes far more sense (we have, of course, imposed the larger boundaries). Our oldest car is now dated at 1910. 25% of cars are from the previous century, the majority of cars 8+ years old, and a quartile within 8 years old.

## Exploring auto data by Brand

Next we'll hop into the attributes of the data based on the car make / brand. 

In [112]:
# Display total number of brands represented in listings
print('Number of auto brands in listings: {} \n'
      .format(autos['brand'].nunique()))

# Display top 20 brands by percentage of total listings
print(autos['brand'].value_counts(normalize=True).head(20), '\n')

# Display share of listings of top 20 brands
top_20_brands = autos['brand'].value_counts(normalize=True).head(20)
print('Share of listings from top 20 brands: {}'
      .format(top_20_brands.sum()))

# Display share of listings of top 5 brands
print('Share of listings from top 5 brands: {}'
      .format(top_20_brands.head().sum()))

Number of auto brands in listings: 40 

volkswagen        0.211404
bmw               0.110179
opel              0.107245
mercedes_benz     0.096652
audi              0.086771
ford              0.069835
renault           0.047075
peugeot           0.029858
fiat              0.025608
seat              0.018252
skoda             0.016418
nissan            0.015339
mazda             0.015231
smart             0.014196
citroen           0.014045
toyota            0.012793
hyundai           0.010010
sonstige_autos    0.009536
volvo             0.009126
mini              0.008802
Name: brand, dtype: float64 

Share of listings from top 20 brands: 0.9283741801863997
Share of listings from top 5 brands: 0.6122497411114947


Here's a readout of the top 20 auto brands by their incidence of listings within our dataset. As we can see, though there are 40 brands listed in total, these 20 represent nearly 93% of listings. It stands to reason that the majority of listings (including each of the top 5) are on behalf of German makers (given that our dataset represents German eBay listings), and the majority of the remainder are of European make. 

Next we'll take a look at mean price of listings by brand.

In [85]:
# Calculate mean price by brand
brand_price = dict()
brands = autos['brand'].value_counts().head(20).index
for brand in brands:
    brand_sum = autos.loc[(autos['brand'] == brand), 'price'].sum()
    brand_count = autos.loc[autos['brand'] == brand, 'brand'].count()
    brand_price[brand] = brand_sum / brand_count

sorted_prices = sorted(list(brand_price.items()), 
                       key=lambda tup: tup[1], reverse=True)    
    
for tup in sorted_prices:
    print('{}: ${}'.format(tup[0], round(tup[1], 2)))

sonstige_autos: $12784.56
mini: $10639.45
audi: $9380.72
mercedes_benz: $8672.65
bmw: $8381.68
skoda: $6409.61
volkswagen: $5436.95
hyundai: $5411.08
toyota: $5167.09
volvo: $4993.21
nissan: $4756.66
seat: $4433.42
mazda: $4129.77
citroen: $3796.26
ford: $3779.27
smart: $3596.4
peugeot: $3113.86
opel: $3005.5
fiat: $2836.87
renault: $2496.07


Our mean prices by brand readout (sorted descending) yields some interesting results. Firstly "sonstige_autos" as our top brand warranted some additional research- it turns out that there is no such brand, and "sonstige" translates to "other". In this sense, this categorization is ostensibly a catch-all for everything not fitting into our 40 total brands, perhaps because of another pick-list. It's a worthy assumption that these may be more luxury autos that fall outside of the more common brands, especially since this category comprises ~1% of listings.

Beyond this, there seems to be a general tendency for more luxury-oriented brands to reside toward the higher end in price.

Let's next expand our analysis of top brands to any auto maker with more than 5% of the share of listings. 

In [114]:
top_5pct_brands = top_20_brands[top_20_brands > .05].index
for brand in sorted(top_5pct_brands, 
                    key=lambda brand: brand_price[brand], reverse=True):
    print('{}: ${}'.format(brand, round(brand_price[brand], 2)))

audi: $9380.72
mercedes_benz: $8672.65
bmw: $8381.68
volkswagen: $5436.95
ford: $3779.27
opel: $3005.5


At a glance, Ford has also made this threshold- this could be explained in that it's also the top-selling American car brand. Audi, Mercedes, and BMW are notably more expensive, whereas Volkswagen fits into a mid-range, and Ford and Opel are our more "affordable" brands.

## Exploring mileage among auto brands

Next we'll review the mileage among our top auto brands. We'll kick off with a process to convert the respective mean mileage and price to dictionaries, then convert these dictionaries to pandas Series, then consolidating these to a pandas DataFrame.

In [150]:
# Construct dictionary for mean mileage of our top auto brands
top_5pct_brands_mileage = dict()
for brand in top_5pct_brands:
    mean_mileage = autos.loc[autos["brand"] == brand, "odometer_km"].mean()
    top_5pct_brands_mileage[brand] = mean_mileage

# Construct dictionary for mean prices of our top auto brands
top_5pct_brands_prices = dict()
for brand in top_5pct_brands:
    top_5pct_brands_prices[brand] = brand_price[brand]

# Convert dictionaries to Series
brand_prices_series = pd.Series(top_5pct_brands_prices)
print(brand_prices_series, '\n')

brand_mileage_series = pd.Series(top_5pct_brands_mileage)
print(brand_mileage_series)

# Convert Series to a unified DataFrame
top_5pct_autos = pd.DataFrame(brand_prices_series, columns=['mean_price'])
top_5pct_autos['mean_mileage'] = brand_mileage_series
top_5pct_autos.sort_values('mean_mileage', ascending=False)

volkswagen       5436.950097
bmw              8381.677306
opel             3005.496077
mercedes_benz    8672.654241
audi             9380.718548
ford             3779.269076
dtype: float64 

volkswagen       128799.877539
bmw              132695.320149
opel             129384.429692
mercedes_benz    131025.669643
audi             129245.400298
ford             124277.108434
dtype: float64


,mean_price,mean_mileage
bmw,8381.677306,132695.320149
mercedes_benz,8672.654241,131025.669643
opel,3005.496077,129384.429692
audi,9380.718548,129245.400298
volkswagen,5436.950097,128799.877539
ford,3779.269076,124277.108434


It seems that all of our top brands have higher mileages and are closely grouped. This is of course a bit distorted by the upper bound of 150,000km and its high incidence. There's a bit of a trend of the higher priced cars having relatively higher mileage, and also that the "foreign" Ford cars have the lowest mileage.

## Further analysis

We've completed some initial cleaning and analysis on columns, prices, mileage, registration dates, and crawler behavior. Some areas for potential additional cleaning and analysis:
* translating German categorical data to English
* uniform formatting of dates
* analysis on models of cars- most common models, etc.
* correlations between mileage and price on listings
* damage as a factor in price, or a function of brand / model